# Importações

In [ ]:
import pandas as pd
import os

# Visualização inicial

Foram fornecidos dois arquivos CSV de dados, um nomeado "amostra_total.csv" e outro "amostra_menor.csv". Será visualizado inicialmente o conteúdo de cada um (devem estar na mesma pasta que este arquivo para a execução).

In [ ]:
# Inicialização dos dataframes (leitura dos csv's)
total_sample = pd.read_csv("./amostra_total.csv", sep=";")
smaller_sample = pd.read_csv("./amostra_menor.csv", sep=";")

In [ ]:
# Visualização das primeiras 5 linhas do arquivo "amostra_total.csv"
total_sample.head()

In [ ]:
# Visualização das primeiras 5 linhas do arquivo "amostra_menor.csv"
smaller_sample.head()

# Dados recebidos

Como observado na visualização inicial acima, supõe-se que foram recebidos dados referentes a:

- Logradouros do Rio de Janeiro (colunas "LOGRADOURO" e "NUMERO");
- Rotas de leiturização às quais os logradouros pertencem (coluna "CODIGO_ROTA");
- Ordem ou índice do logradouro em determinada rota (coluna "SEQUENCIA");
- Identificador único de cada logradouro em uma rota (coluna "INDICE");
- Coordenadas geográficas dos logradouros (colunas "LATITUDE" e "LONGITUDE").

De imediato, é necessário estudar se há dados contidos na amostra menor que não estão presentes na amostra total, se existe algum identificador repetido na coluna "INDICE", procurar e tratar dados nulos e realizar a conversão de tipos nas colunas "INDICE", "SEQUENCIA" e "NUMERO", de float/object (string) para integer.

# Comparação entre a amostra total e a menor

In [ ]:
# Comparação de tamanhos das amostras de dados
total_rows         = int(total_sample.count().max())
total_rows_smaller = int(smaller_sample.count().max())

print(f"A amostra total contém {total_rows} linhas, enquanto a amostra menor contém {total_rows_smaller}.")
print(f"Dessa forma, há uma diferença de {total_rows - total_rows_smaller} linhas.")

In [ ]:
# Teste para avaliar se a amostra menor está contida na total ou se são outros dados
df_test             = pd.merge(total_sample, smaller_sample, "left", "INDICE")
total_plus_added  = int(df_test.count().max())

if total_plus_added - total_rows == 0:
    print("Todos os dados da amostra menor estão contidos na amostra total, dado que o merge entre os dois datasets não resultou em nenhuma linha adicional.")
else:
    print(f"Nem todos os dados da amostra menor estão contidos na amostra total, dado que o merge entre os dois datasets resultou em {total_plus_added - total_rows} linha(s) adicional(is).")

### Dado que todas as informações da amostra menor estão presentes na amostra total, ela não será considerada nas análises futuras.

# Presença de identificadores duplicados

In [ ]:
# Procura por identificadores publicados
duplicated_identifiers_mask = total_sample["INDICE"].duplicated() == True
if duplicated_identifiers_mask.any():
    print(f"Há {total_sample[duplicated_identifiers_mask].count().max()} identificadores duplicados nos dados.")
else:
    print("Não há identificadores duplicados nos dados.")

# Dados nulos

In [ ]:
# Checagem por dados nulos
null_mask       = total_sample.isnull().any(axis=1)
null_rows       = total_sample[null_mask].count().max()

print(f"Há {null_rows} linhas com dados nulos na amostra total")

In [ ]:
# Visualização dos dados nulos
total_sample[null_mask].head()

### Ao que tudo indica, os dados nulos se referem a logradouros sem número registrado. Essa hipótese é verificada abaixo.

In [ ]:
# Visualização de registros nulos em cada coluna
for col, null_count in total_sample[null_mask].count().items():
    if null_rows == null_count:
        print(f'A coluna "{col}" não contém registros nulos.')
    else:
        print(f'A coluna "{col}" possui {null_rows - null_count} registros nulos.')

### Como a coluna "NUMERO" é a única com dados nulos, ou seja, mesmo sem número ainda se tem todos os outros dados relativos aquela rota e aquele logradouro, por ora os valores nulos serão substituídos por 0. Afinal, essa ação é necessária também para a conversão de tipos mencionada anteriormente.

In [ ]:
# Trocando valores nulos por 0
total_sample.fillna(0, inplace=True)

# Reavaliando a máscara para filtrar o dataframe por dados nulos
null_mask = total_sample.isnull().any(axis=1)

if int(total_sample[null_mask].count().max()) > 0:
    print("Mesmo após substituição, o dataset ainda contém dados nulos.")
else:
    print("Todos os valores nulos foram substituídos com sucesso.")

# Conversão de tipos

In [ ]:
# Checagem do tipo da coluna "NUMERO" da amostra, pois não aparentava ser float na visualização inicial (pelo menos não na amostra total)
print(total_sample["NUMERO"].dtype)

### Visto que o tipo da coluna "NUMERO" é object (string), provavelmente há registros fora do padrão que contêm letras. Esses registros são tratados abaixo.

Alguns números foram registrados com complementos, então será criada uma coluna de complemento para armazenar esses dados.

In [ ]:
# Ambas as funções retornam strings pois os dados da coluna na qual serão aplicadas estão neste formato
def get_adress_number_and_complement(s : pd.Series) -> list[str, str]:
    """
    Recebe um registro do dataset das colunas "NUMERO" e "COMPLEMENTO" e retorna o número
    devidamente processado (ou seja, com letras e caracteres não-numéricos retirados) e o
    complemento do endereço caso esteja contido na coluna "NUMERO". Caso contrário, o
    complemento é retorado como "N" (de "Não").

    Parâmetros:
    - s: pd.Series / registro das colunas "NUMERO" e "COMPLEMENTO".

    Retorno:
    - [número sem caracteres não-numéricos, complemento].
    """
    try:
        return [str(int(s["NUMERO"])), "N"]
    except:
        return get_number_and_complement_within_str(s["NUMERO"].strip())


def get_number_and_complement_within_str(s : str) -> list[str, str]:
    """
    Recebe uma string e tenta retornar um número nela contido
    e o restante da string como complemento. Caso não encontre um
    número, retorna ["0", restante da string]. Caso não haja uma
    parte restante da string, essa é substituída no retorno por "N"
    (de "Não").

    Parâmetros:
    - s: string que supostamente contém um número.

    Retorno:
    - [número contido na string enviada como parâmetro, restante da string].
    """
    found = False
    number = ""
    initial_index = 0
    for i in range(len(s)):
        try:
            number += str(int(s[i]))
            if not found:
                initial_index = i                
            found = True
        except:
            if found:
                return [number, s[0:initial_index] + s[i:]]
            continue
    
    return [number, s[0:initial_index]] if number != "" else ["0", "N"] 

In [ ]:
# Tratamento das strings na coluna "NUMERO"
total_sample["COMPLEMENTO"] = ""
total_sample[["NUMERO", "COMPLEMENTO"]] = total_sample[["NUMERO", "COMPLEMENTO"]].apply(get_adress_number_and_complement, axis=1, result_type="broadcast")

In [ ]:
# Conversão de float para integer
total_sample["INDICE"]     = total_sample["INDICE"].astype(int)
total_sample["SEQUENCIA"]  = total_sample["SEQUENCIA"].astype(int)
total_sample["NUMERO"]     = total_sample["NUMERO"].astype(int)

for col in total_sample.columns:
    print(f'Tipo da coluna "{col}": {total_sample[col].dtype}.')

In [ ]:
# Visualização dos dados com tipos convertidos
total_sample.head()

# Ordenação dos dados

A partir desse tratamento inicial, ordena-se os dados de acordo com as colunas "CODIGO_ROTA" e "SEQUENCIA", a fim de organizar as informações, visualizar o trajeto de cada rota de forma ordenada e possivelmente gerar insights para próximos passos.

In [ ]:
# Ordenação dos dados de acordo com as colunas mencionadas
total_sample = total_sample.sort_values(by=["CODIGO_ROTA", "SEQUENCIA"]).reset_index(drop=True)

In [ ]:
# Visualização dos dados ordenados
total_sample.head(10)

# Registros duplicados

Apesar dos identificadores únicos não apresentarem duplicatas, observa-se após a ordenação registros que indicam a existência de duplicatas nos dados, dado que o código da rota, o logradouro e o número do endereço são idênticos em mais de uma linha. O mais alarmante é que existem registros em que, além dos dados mencionados, o número de sequência também é igual, o que indica uma possível reinserção de dados idênticos.

Essa hipótese é verificada a seguir.

In [ ]:
# Verificação da existência de registros supostamente duplicados
duplicated_mask = total_sample[["CODIGO_ROTA", "LOGRADOURO", "NUMERO"]].duplicated(keep="last") == True
duplicated_rows = int(total_sample[duplicated_mask].count().max())
total_rows      = int(total_sample.count().max())
percentage = "%.2f" % (duplicated_rows / total_rows * 100)

if duplicated_rows > 0:
    print(f"Há {duplicated_rows} registros duplicados nos dados. Ou seja, {percentage}% dos dados ({duplicated_rows} registros duplicados para {total_rows} registros totais).")
else:
    print("Não há registros duplicados nos dados.")

### Hipótese: dados duplicados se referem a registros de hidrômetros localizados em um mesmo logradouro, fenômeno observado em sobrados e prédios por exemplo

Dessa forma, será criada uma coluna que registra a quantidade de hidrômetros em cada logradouro. Antes, porém, é necessário avaliar se as distâncias registradas nas colunas "LATITUDE" e "LONGITUDE" são significativas, o que poderia negar a hipótese acima. Esse processo é realizado a seguir.

In [ ]:
# Verificação do desvio padrão da latitude e longitude dos dados repetidos
coordinates_std = total_sample.groupby(["CODIGO_ROTA", "LOGRADOURO", "NUMERO"])[["LATITUDE", "LONGITUDE"]].std().reset_index()
all_rows = int(coordinates_std.count().max())

std_greater_than_zero = (coordinates_std[["LATITUDE", "LONGITUDE"]] > 0).any(axis=1)
gt_zero_rows = int(coordinates_std[std_greater_than_zero].count().max())

coordinates_std[std_greater_than_zero][["LATITUDE", "LONGITUDE"]].describe()

In [ ]:
# Verificação do desvio padrão da latitude e longitude dos dados totais
coordinates_std[["LATITUDE", "LONGITUDE"]].describe()

### Observa-se que pelo menos 75% dos dados duplicados que apresentam um desvio padrão maior que 0 nas medidas de coordenadas geográficas (latitude e longitude) não ultrapassam o limite de 0.00007 nesse desvio padrão, o que representa uma diferença insignificante e corrobora a hipótese apresentada.

Essa afirmação é reforçada pelo fato de que, ao analisar-se o desvio padrão para essas medidas com os dados totais (não só os duplicados), pelo menos 75% não ultrapassa o limite de 0.000055 no desvio padrão.

Entretanto, ainda há um volume desses dados duplicados (25% no máximo) que apresenta um desvio padrão significativo (considerou-se maior que 0.0001 como significativo) e que deve ser verificado.

In [ ]:
# Verificação do volume de dados com desvio padrão significativo
coordinates_std_mask = (coordinates_std[["LATITUDE", "LONGITUDE"]] > 0.0001).any(axis=1)
relevant_rows = int(coordinates_std[coordinates_std_mask].count().max())
percentage = "%.2f" % (relevant_rows / gt_zero_rows * 100)

print(f"""Dos dados duplicados que apresentam um desvio padrão maior que 0 nas medidas de coordenadas geográficas (latitude e longitude),
{percentage}% tem valor significativo de desvio padrão ({relevant_rows} registros significativos para {gt_zero_rows} registros analisados).

Em comparação com os dados totais (agrupadas as duplicatas), esses registros significativos representam {"%.2f" % (relevant_rows / all_rows * 100)}%
({relevant_rows} registros significativos para {all_rows} registros totais).""")

### Dado o baixo volume de registros com desvio padrão relevante, por ora, será considerado que a diferença observada entre os registros de coordenadas geográficas para dados duplicados é irrelevante, possibilitando a criação da coluna de quantidade de hidrômetros e o descarte dos dados duplicados.

In [ ]:
# Criando a coluna de quantidade de hidrômetros
hydrometer_amount = total_sample.groupby(["CODIGO_ROTA", "LOGRADOURO", "NUMERO"]).size().reset_index().rename(columns={0: "QUANTIDADE_HIDROMETROS"})

### Pressupondo que os dados duplicados são dispensáveis, uma vez que se tem as quantidades de hidrômetros para cada logradouro, os mesmos são descartados a seguir.

In [ ]:
# Descartando dados duplicados e adicionando a coluna de quantidade de hidrômetros
sample_with_quantity = total_sample.drop(total_sample[duplicated_mask].index).merge(hydrometer_amount, "inner", ["CODIGO_ROTA", "LOGRADOURO", "NUMERO"])
sample_with_quantity.head()

# Tratamento da coluna "SEQUENCIA"

Observa-se que a coluna "SEQUENCIA" possui registros inconsistentes. Por exemplo, a rota "10_14" só possui um registro nos dados recebidos, porém seu número de sequência é 2160, o que indica que deveriam haver outros 2159 registros para essa rota, já que a sequência indica o índice daquele logradouro dentro daquela rota (ou seja, a ordem de leiturização).

Além disso, há rotas com diferentes endereços possuindo o mesmo número de sequência.

Por fim, com a limpeza de dados duplicados, algumas sequências apresentam saltos em algumas rotas, e não uma ordem de crescimento linear, o que também precisa ser corrigido.

Dessa forma, o número de sequência dos registros será recomputado para cada rota, a fim de torná-lo coerente com os registros presentes.

In [ ]:
# Tratando valores de sequência que não começam em 0 como o caso da rota "10_14", valores duplicados de sequência e saltos na sequência
routes = sample_with_quantity.groupby("CODIGO_ROTA")
sample_with_quantity["SEQUENCIA"] = routes.cumcount()

In [ ]:
# Verificando se ainda existem valores incoerentes
if sample_with_quantity[["CODIGO_ROTA", "SEQUENCIA"]].duplicated().any():
    print("Ainda há valores duplicados de sequência.")
else:
    print("Todos os valores duplicados de sequência foram eliminados.")

sample_with_quantity.head(10)

# Finalização da análise e do tratamento

Com todas as etapas de tratamento realizadas acima, possibilitadas pela análise feita, conclui-se o o processo de análise e tratamento inicial dos dados, objetivo deste Jupyter Notebook. Por fim, os dados tratados são exportados para o caminho "./dados_tratados/dados_tratados.csv".

In [ ]:
if not os.path.exists("./dados_tratados/"):
    os.mkdir("./dados_tratados/")

sample_with_quantity.to_csv("./dados_tratados/dados_tratados.csv")

# Informações fornecidas pelo parceiro após reunião

Após reunião de alinhamento acerca do projeto, os parceiros de negócios ratificaram todas as hipóteses supramencionadas e apontaram que o tratamento realizado nos dados está, em sua maioria, correto. O único ponto de ajuste seria o descarte dos dados duplicados, que não pode ser feito de forma irreversível, pois as informações únicas de cada rota (ou seja, seu identificador único) devem estar presentes em um possível output. Ou seja, esse descarte será feito de forma temporária apenas para a etapa de processamento e utilização dos dados pelo backend do projeto, provendo uma execução otimizada dos algoritmos, e será retornado ao frontend uma versão expandida do output do backend, incluindo dados previamente descartados.

Dessa forma, como próxima etapa de trabalho com os dados recebidos, será estruturada uma pipeline de tratamento padronizada e generalizada, para que assim a aplicação desenvolvida possa receber diferentes inputs de dados no formato CSV (conservando, claro, a estrutura de dados existente) e gerar outputs diferenciados para cada amostra.

Além disso, segundo o parceiro, serão enviados novos dados idênticos aos já recebidos, porém contendo também o complemento dos endereços como uma nova coluna do dataset.

# Recebimento de novos dados: coordenadas geográficas das bases

Foram recebidas informações a respeito das coordenadas geográficas (latitude e longitude) das duas bases da Aegea Saneamento na cidade do Rio de Janeiro, ou seja, os locais que representam o início do percurso de todos os leituristas, pois é de onde saem para realizar seu trabalho.
Esses dados serão mostrados abaixo, mas é necessário mencionar que eles não serão incluídos na etapa inicial do desenvolvimento do projeto por conta do aumento do escopo que acompanha essa mudança.

In [ ]:
# Lendo as coordenadas geográficas das bases (passadas por um arquivo TXT que deve estar na mesma pasta que este arquivo para a execução)
bases = open("bases.txt")

print("Coordenadas geográficas das bases\n" + bases.read())